# 📓 07_transform_playlist_items_manual_static.ipynb

## Objetivo

Construir la relación actual entre
playlist manual ↔ video,
usando la API de YouTube, y guardarla como tabla estática.

-1 fila = 1 video dentro de 1 playlist manual  
-estado actual  
-se sobrescribe  
-NO histórico todavía 

In [ ]:
from pathlib import Path
import pandas as pd
import requests
import os
from datetime import datetime
from dotenv import load_dotenv


In [ ]:
load_dotenv()

API_KEY = os.getenv("YOUTUBE_API_KEY")
assert API_KEY, "Falta YOUTUBE_API_KEY"


In [ ]:
PROJECT_ROOT = Path.cwd().parents[0]

PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)


In [ ]:
df_playlists_manual_static = pd.read_parquet(
    PROCESSED_PATH / "playlists_manual_static.parquet"
)

df_playlists_manual_static[["playlist_id", "title"]].head()


In [ ]:
from datetime import datetime, timezone

# Timestamp explícitamente en UTC
EXTRACTED_AT = datetime.now(timezone.utc)

EXTRACTED_AT

In [ ]:
rows = []

for _, playlist in df_playlists_manual_static.iterrows():
    playlist_id = playlist["playlist_id"]

    page_token = None

    while True:
        params = {
            "part": "snippet,contentDetails",
            "playlistId": playlist_id,
            "maxResults": 50,
            "key": API_KEY
        }

        if page_token:
            params["pageToken"] = page_token

        response = requests.get(
            "https://www.googleapis.com/youtube/v3/playlistItems",
            params=params
        )
        response.raise_for_status()

        data = response.json()

        for item in data.get("items", []):
            added_at_peru = (
                pd.to_datetime(item["snippet"]["publishedAt"], utc=True)
                .tz_convert("America/Lima")
                .tz_localize(None)
            )

            rows.append({
                "playlist_id": playlist_id,
                "video_id": item["contentDetails"]["videoId"],
                "position": item["snippet"]["position"],
                "added_at": added_at_peru,
                "extracted_at": EXTRACTED_AT
            })

        page_token = data.get("nextPageToken")
        if not page_token:
            break


In [ ]:
df_playlist_items_manual_static = pd.DataFrame(rows)

df_playlist_items_manual_static.head()

In [ ]:
df_playlist_items_manual_static.shape

In [ ]:
df_playlist_items_manual_static.dtypes

In [ ]:
# Convertir added_at a datetime UTC
df_playlist_items_manual_static["added_at"] = (
    pd.to_datetime(df_playlist_items_manual_static["added_at"], utc=True)
)


In [ ]:
df_playlist_items_manual_static.dtypes

In [ ]:
df_playlist_items_manual_static = df_playlist_items_manual_static[
    [
        "playlist_id",
        "video_id",
        "position",
        "added_at",
        "extracted_at",
    ]
]

> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.

In [ ]:
output_file = PROCESSED_PATH / "playlist_items_manual_static.parquet"

df_playlist_items_manual_static.to_parquet(output_file, index=False)

output_file


In [ ]:
df = pd.read_parquet(
    PROCESSED_PATH / "playlist_items_manual_static.parquet"
)

df.head()
